In [3]:

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, brier_score_loss, mean_squared_error
from sklearn.calibration import CalibrationDisplay
import numpy as np
import matplotlib.pyplot as plt

In [4]:
YEARS = [2018, 2019, 2020, 2021, 2022, 2023,2024]

# %%
data_all = pd.DataFrame()

def calculate_seconds(row):
    if row['qtr'] != 5:
        return 3600 - row['game_seconds_remaining']
    else:
        return 600 - row['game_seconds_remaining'] + 3600


def get_quarter_value(dataf):
    if 'END QUARTER' in dataf['desc']:
        return dataf['level_0']
    else:
        return None

for i in YEARS:  
    i_data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(i) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data_all = pd.concat([data_all,i_data])

ppr = 1

data = data_all.loc[data_all.season_type=='REG']
#data = data_all.loc[(data_all.play_type.isin(['no_play','pass','run'])) & (data_all.epa.isna()==False)]
#data.loc[data['pass']==1, 'play_type'] = 'pass'
#data.loc[data.rush==1, 'play_type'] = 'run'
data.reset_index(drop=True, inplace=True)
data['turnover'] = data['interception'] + data['fumble_lost']
data = data.dropna(subset=['posteam'])
data['inside_10'] = (data['yardline_100'] < 10).astype(int)
data['20+_play'] = (data['yards_gained'] > 19).astype(int)
data['short_pass'] = (data['air_yards'] < 10).astype(int)
data['medium_pass'] = ((data['air_yards'] > 9)&(data['air_yards']<20)).astype(int)
data['deep_pass'] = (data['air_yards'] > 19).astype(int)
data['end_zone_target'] = (data['yardline_100'] - data['air_yards']) <= 0
data['fantasy_points'] = (
    data['complete_pass'] * ppr +          # 1 point per completion
    data['touchdown'] * 6 +           # 6 points per touchdown
    data['yards_gained'] * 0.1        # 0.1 points per yard gained
)
data['distance_to_EZ_after_target'] = data['yardline_100'] - data['air_yards']


MemoryError: Unable to allocate 386. KiB for an array with shape (49434,) and data type int64

In [ ]:
def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [ ]:
    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]


    # derive implied team total from betting market data
    data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
    data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)

    # Use list comprehension with zip for more efficient row-wise operations
    data['implied_posteam_total'] = [
    total_finder(has_ball, home_number, away_number)
        for has_ball, home_number, away_number in zip(data['posteam_type'], data['home_implied_total'], data['away_implied_total'])
]

    

In [ ]:
    
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


In [ ]:
df

In [ ]:


# Load and prepare data (same as before)
predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'season', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]
X = df[predictors]
y = df['pass_touchdown']

# Convert categorical variables
X = pd.get_dummies(X, columns=['pass_location'], drop_first=True)
X.fillna(X.mean(), inplace=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create XGBoost regression model with probability-focused objective
model = xgb.XGBRegressor(
    objective='reg:logistic',  # This constrains outputs to [0,1]
    eval_metric=['rmse', 'logloss'],  # Track both regression and classification metrics
)

# Hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 3]
}

# Grid search with both MSE and ROC-AUC scoring
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=['neg_mean_squared_error', 'roc_auc'],
    refit='neg_mean_squared_error',  # Choose MSE as primary metric
    cv=5
)
grid_search.fit(X_train, y_train)

# Best model
touchdown_model = grid_search.best_estimator_

# Make predictions
y_pred = touchdown_model.predict(X_test)

# Evaluate both regression and classification metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
roc_auc = roc_auc_score(y_test, y_pred)

print("Model Evaluation:")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"RMSE: {rmse:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

# Check calibration across probability ranges
prob_ranges = np.linspace(0, 1, 11)
print("\nProbability Calibration Check:")
for i in range(len(prob_ranges)-1):
    mask = (y_pred >= prob_ranges[i]) & (y_pred < prob_ranges[i+1])
    if mask.any():
        avg_pred_prob = y_pred[mask].mean()
        actual_prob = y_test[mask].mean()
        n_samples = mask.sum()
        print(f"\nProbability range {prob_ranges[i]:.1f}-{prob_ranges[i+1]:.1f} (n={n_samples}):")
        print(f"Average predicted: {avg_pred_prob:.3f}")
        print(f"Actual observed: {actual_prob:.3f}")

# Feature importance
importance = touchdown_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance_df)

# Example prediction function
def predict_touchdown_probability(model, play_data):
    """
    Make touchdown probability predictions for new plays.
    """
    # Ensure play_data has same preprocessing as training data
    play_data = pd.get_dummies(play_data, columns=['pass_location'], drop_first=True)
    
    # Add any missing columns from training data
    for col in model.feature_names_in_:
        if col not in play_data.columns:
            play_data[col] = 0
            
    # Reorder columns to match training data
    play_data = play_data[model.feature_names_in_]
    
    # Get predictions and ensure they're in [0,1]
    probabilities = model.predict(play_data)
    probabilities = np.clip(probabilities, 0, 1)
    
    return probabilities

# Example usage with detailed probability analysis
example_play = pd.DataFrame({
    'air_yards': [20],
    'yardline_100': [70],
    'ydstogo': [10],
    'down': [1],
    'pass_location': ['middle'],
    'season': [2023],
    'qb_hit': [0],
    'end_zone_target': [0],
    'distance_to_EZ_after_target': [50]
})

prob = predict_touchdown_probability(touchdown_model, example_play)
print(f"\nExample Play Touchdown Probability: {prob[0]:.1%}")

# Print top factors influencing this prediction
def explain_prediction(model, play_data, feature_importance_df):
    """Explain what factors most influenced a specific prediction"""
    # Get the top 5 most important features
    top_features = feature_importance_df.head()
    
    print("\nTop factors influencing this prediction:")
    for _, row in top_features.iterrows():
        feature = row['Feature']
        if feature in play_data.columns:
            value = play_data[feature].iloc[0]
            print(f"{feature}: {value} (importance: {row['Importance']:.3f})")

explain_prediction(touchdown_model, example_play, feature_importance_df)

In [ ]:
import pickle


with open('touchdown_model.pkl', 'wb') as file:
    pickle.dump(touchdown_model, file)